<a href="https://colab.research.google.com/github/alibekk93/soccer_fun_stuff/blob/main/FBRef_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FBRef data getter
https://fbref.com/en/

## Setup

In [ ]:
!pip install requests_html

In [133]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [4]:
session = HTMLSession()

## New Section

In [150]:
def get_team_summary_table(tables:list, table_index:int=0) -> pd.DataFrame():
  """Parses team summary table given a list of all tables from URL
  """
  table = tables[table_index]

  table_headers = [th.text for th in table.find_all('th')][9:41]
  table_headers[-11:] = ['p90_' + th for th in table_headers[-11:]]
  player_names = [th.text for th in table.find_all('th')][42:-2]
  table_data = [th.text for th in table.find_all('td')]
  table_data_cleaned = [r.tolist()[:-1] for r in np.array(table_data).reshape(-1, 33)][:-2]

  summary_df = pd.DataFrame(table_data_cleaned, columns=table_headers)
  summary_df['Player'] = player_names

  summary_df[summary_df['Min']=='']

  for col in summary_df.columns:
    try:
      summary_df.loc[:, col] = summary_df.loc[:, col].astype(float)
    except:
      pass

  return summary_df

In [101]:
r = session.get('https://fbref.com/en/squads/b2b47a98/2022-2023/Newcastle-United-Stats')
soup = BeautifulSoup(r.html.raw_html)

In [102]:
tables = soup.find_all('table')

In [151]:
newcastle_summary = get_team_summary_table(tables)

<ipython-input-150-aeb110eada5a>:19: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  summary_df.loc[:, col] = summary_df.loc[:, col].astype(float)


In [155]:
newcastle_summary

,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,G+A,...,p90_Ast,p90_G+A,p90_G-PK,p90_G+A-PK,p90_xG,p90_xAG,p90_xG+xAG,p90_npxG,p90_npxG+xAG,Player
0,eng ENG,DF,31.0,38.0,38.0,"3,341",37.1,1,7,8,...,0.19,0.22,0.03,0.22,0.02,0.33,0.36,0.02,0.36,Kieran Trippier
1,eng ENG,GK,30.0,37.0,37.0,"3,261",36.2,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Nick Pope
2,ch SUI,DF,30.0,36.0,36.0,"3,206",35.6,1,3,4,...,0.08,0.11,0.03,0.11,0.14,0.09,0.23,0.14,0.23,Fabian Schär
3,nl NED,DF,22.0,36.0,35.0,"3,127",34.7,0,0,0,...,0.00,0.00,0.00,0.00,0.07,0.03,0.10,0.07,0.10,Sven Botman
4,eng ENG,DF,30.0,38.0,35.0,"3,111",34.6,1,0,1,...,0.00,0.03,0.03,0.03,0.06,0.04,0.10,0.06,0.10,Dan Burn
5,br BRA,MF,24.0,32.0,32.0,"2,726",30.3,4,5,9,...,0.17,0.30,0.13,0.30,0.09,0.13,0.23,0.09,0.23,Bruno Guimarães
6,eng ENG,MF,22.0,35.0,31.0,"2,558",28.4,3,6,9,...,0.21,0.32,0.11,0.32,0.25,0.21,0.46,0.25,0.46,Joe Willock
7,br BRA,"MF,FW",25.0,32.0,30.0,"2,661",29.6,6,1,7,...,0.03,0.24,0.20,0.24,0.24,0.08,0.32,0.24,0.32,Joelinton
8,py PAR,FW,28.0,34.0,29.0,"2,491",27.7,11,2,13,...,0.07,0.47,0.40,0.47,0.29,0.11,0.40,0.29,0.40,Miguel Almirón
9,eng ENG,MF,24.0,33.0,28.0,"2,518",28.0,1,4,5,...,0.14,0.18,0.04,0.18,0.13,0.12,0.25,0.13,0.25,Sean Longstaff
